In [73]:
import pandas as pd
import numpy as np

In [74]:
data = pd.read_csv("TripAdvisor_RestauarantRecommendation.csv")

In [75]:
data.head()

,Name,Street Address,Location,Type,Reviews,No of Reviews,Comments,Contact Number,Trip_advisor Url,Menu,Price_Range
0,Betty Lou's Seafood and Grill,318 Columbus Ave,"San Francisco, CA 94133-3908","Seafood, Vegetarian Friendly, Vegan Options",4.5 of 5 bubbles,243 reviews,NaN,+1 415-757-0569,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1,Coach House Diner,55 State Rt 4,"Hackensack, NJ 07601-6337","Diner, American, Vegetarian Friendly",4 of 5 bubbles,84 reviews,"Both times we were there very late, after 11 P...",+1 201-488-4999,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
2,Table Talk Diner,2521 South Rd Ste C,"Poughkeepsie, NY 12601-5476","American, Diner, Vegetarian Friendly",4 of 5 bubbles,256 reviews,Waitress was very friendly but a little pricey...,+1 845-849-2839,https://www.tripadvisor.com//Restaurant_Review...,http://tabletalkdiner.com/menu/breakfast/,$$ - $$$
3,Sixty Vines,3701 Dallas Pkwy,"Plano, TX 75093-7777","American, Wine Bar, Vegetarian Friendly",4.5 of 5 bubbles,235 reviews,Not sure why I went there for the second time....,+1 469-620-8463,https://www.tripadvisor.com//Restaurant_Review...,https://sixtyvines.com/menu/plano-tx/,$$ - $$$
4,The Clam Bar,3914 Brewerton Rd,"Syracuse, NY 13212","American, Bar, Seafood",4 of 5 bubbles,285 reviews,Doesn't look like much from the outside but wa...,+1 315-458-1662,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$


In [76]:
data.columns

Index(['Name', 'Street Address', 'Location', 'Type', 'Reviews',
       'No of Reviews', 'Comments', 'Contact Number', 'Trip_advisor Url',
       'Menu', 'Price_Range'],
      dtype='object')

In [77]:
data['Name'].nunique()

2641

In [78]:
data['Location'].nunique()

2653

In [79]:
data['Type'][6]

' American, Steakhouse, Bar'

In [80]:
#take comments and identify similar kind of restaurants

In [81]:
data['Comments']

0                                                     NaN
1       Both times we were there very late, after 11 P...
2       Waitress was very friendly but a little pricey...
3       Not sure why I went there for the second time....
4       Doesn't look like much from the outside but wa...
                              ...                        
3057    We had to wait a few minutes to get it but it ...
3058    Came here to see Andrea Gibson perform, we wer...
3059    We come to Brio once a month. They are good fo...
3060    Stopped in yesterday for Breakfast. When in Je...
3061    First time for dinner. 2 couples. Delicious mu...
Name: Comments, Length: 3062, dtype: object

In [82]:
data2 = data[["Name","Comments"]]

In [83]:
indices = pd.Series(data2.index, index = data2['Name'])

In [84]:
data2.head()

,Name,Comments
0,Betty Lou's Seafood and Grill,NaN
1,Coach House Diner,"Both times we were there very late, after 11 P..."
2,Table Talk Diner,Waitress was very friendly but a little pricey...
3,Sixty Vines,Not sure why I went there for the second time....
4,The Clam Bar,Doesn't look like much from the outside but wa...


In [85]:
data2.isna().sum()

Name          0
Comments    615
dtype: int64

In [86]:
data2.shape

(3062, 2)

In [87]:
data2 = data2.dropna()
data2.isna().sum()

Name        0
Comments    0
dtype: int64

In [88]:
data2.shape

(2447, 2)

In [89]:
feature = data2['Comments'].tolist()

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
tfidf = TfidfVectorizer(input=feature, stop_words = "english")

In [92]:
tfidf_matrix = tfidf.fit_transform(feature)

In [93]:
tfidf_matrix.shape

(2447, 6245)

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [96]:
def restaurant_rec_sys(name, similarity = similarity):
    index = indices[name]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key = lambda x: x[1],reverse = True)
    similarity_scores = similarity_scores[:11]
    res_indices = [i[0] for i in similarity_scores]
    return data2["Name"].iloc[res_indices]

In [98]:
restaurant_rec_sys("Sixty Vines", similarity= similarity)

4                         The Clam Bar
1075    Toulouse Cafe and Bar (Dallas)
1842                          Lapellah
1156               LongHorn Steakhouse
2753        Copper Top Tavern Camillus
2018               Terra Mediterranean
2194                      P.F. Chang's
741                Shanghai You Garden
744           Gringo's Mexican Kitchen
2416                 Logan's Roadhouse
1648                        Applebee's
Name: Name, dtype: object